In [371]:
from pandas.core.common import SettingWithCopyWarning
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm,t,multivariate_t
from sklearn.linear_model import LinearRegression
import sys
import warnings
warnings.filterwarnings(action='ignore',category=FutureWarning)
warnings.filterwarnings(action='ignore',category=SettingWithCopyWarning)


# Problem 1

In [315]:
P_t = 30.0
r_sigma = 0.5
r_simulate = np.random.normal(loc=0,scale=r_sigma,size=2000)
P_t1_discrete = P_t * (1 + r_simulate)
P_t1_log = P_t * np.exp(r_simulate)
P_t1_classic = P_t + r_simulate


In [329]:
print(f'Classic & {P_t1_classic.mean()} & {P_t1_classic.std()} \\\\')
print(f'Discrete & {P_t1_discrete.mean()} & {P_t1_discrete.std()} \\\\')
print(f'Geometric & {P_t1_log.mean()} & {P_t1_log.std()} \\\\')

Classic & 29.99489568675101 & 0.49704791031134987 \\
Discrete & 29.84687060253026 & 14.911437309340497 \\
Geometric & 33.79028077895063 & 18.188632338032303 \\


In [331]:
print(f'Classic & {P_t} & {r_sigma} \\\\')
print(f'Discrete & {P_t} & {np.sqrt(P_t)*r_sigma} \\\\')
print(f'Geometric & {P_t*np.exp(r_sigma*r_sigma)} & {P_t*np.sqrt((np.exp(2*r_sigma*r_sigma) -np.exp(r_sigma*r_sigma)))} \\\\')

Classic & 30.0 & 0.5 \\
Discrete & 30.0 & 2.7386127875258306 \\
Geometric & 38.520762500632244 & 18.11701599632644 \\


# Problem 2

In [332]:
price_data = pd.read_csv("DailyPrices.csv")

In [333]:
# the return_calculate function:
def return_calculate(df, method="DISCRETE", dateColumn="Date"):
    vars = list(df.columns.values)
    var_num = len(vars)
    if dateColumn not in vars:
        raise ValueError("dateColumn: "+str(dateColumn)+" not in DataFrame")
    vars.remove(dateColumn)


    var_num -= 1
    price = df[vars].values
    price_2 = price[1:]/price[:-1]


    if method.upper() == "DISCRETE":
        price_2 = price_2 - 1
    elif method.upper() == "LOG":
        price_2 = np.log(price_2)
    elif method.upper() == "CLASSIC":
        price_2 = price[1:] - price[:-1]
    else:
        raise ValueError("method: ", method, " must be in (\"LOG\",\"DISCRETE\")")

    dates = df[dateColumn].values[1:]
    result =pd.concat ([pd.DataFrame({dateColumn:dates}),pd.DataFrame(columns=vars, data=price_2)],axis=1)
    return result


# calculate the weight terms
def cum_weight(n, lambda_):
    w = np.zeros(n)
    cum_w = np.zeros(n)
    for i in range(0, n):
        w[n - i - 1] = (1 - lambda_) * (np.power(lambda_, i + 1))

    tw = w.sum()
    for i in range(0, n):
        w[n - i - 1] = w[n - i - 1] / tw

    return w


# calculate cumulative variance for each (x,y) pair
def cum_var(x, y, lambda_):
    n = len(x)
    w = cum_weight(n, lambda_=lambda_)
    x_bar = np.mean(x)
    y_bar = np.mean(y)


    cov = np.sum(w*((x-x_bar)*(y-y_bar)))

    return cov


# calculate the covariance matrix
def ew_cov(df, lambda_):
    n = df.shape[1]
    df = np.transpose(df)
    cov_m = np.zeros((n, n))
    for i in range(0, n):
        for j in range(0, n):
            cov = cum_var(df[i], df[j], lambda_=lambda_)
            cov_m[i, j] = cov

    return cov_m



In [184]:
# calculate the returns for all stocks:
classic_return = return_calculate(price_data,method="CLASSIC", dateColumn="Date")
discrete_return = return_calculate(price_data,method="DISCRETE", dateColumn="Date")
log_return = return_calculate(price_data,method="LOG", dateColumn="Date")

In [405]:
discrete_return.head(10)

,Date,SPY,AAPL,MSFT,AMZN,TSLA,GOOGL,GOOG,META,NVDA,...,PNC,MDLZ,MO,ADI,GILD,LMT,SYK,GM,TFC,TJX
0,2/15/2022 0:00,0.016127,0.023152,0.018542,0.008658,0.053291,0.007987,0.008319,0.015158,0.091812,...,0.012807,-0.004082,0.004592,0.052344,0.003600,-0.012275,0.033021,0.026240,0.028572,0.013237
1,2/16/2022 0:00,0.001121,-0.001389,-0.001167,0.010159,0.001041,0.008268,0.007784,-0.020181,0.000604,...,0.006757,-0.002429,0.005763,0.038879,0.009294,0.012244,0.003363,0.015301,-0.001389,-0.025984
2,2/17/2022 0:00,-0.021361,-0.021269,-0.029282,-0.021809,-0.050943,-0.037746,-0.037669,-0.040778,-0.075591,...,-0.034949,0.005326,0.015017,-0.046988,-0.009855,0.004833,-0.030857,-0.031925,-0.033380,-0.028763
3,2/18/2022 0:00,-0.006475,-0.009356,-0.009631,-0.013262,-0.022103,-0.016116,-0.013914,-0.007462,-0.035296,...,-0.000646,-0.000908,0.007203,-0.000436,-0.003916,-0.005942,-0.013674,-0.004506,-0.003677,0.015038
4,2/22/2022 0:00,-0.010732,-0.017812,-0.000729,-0.015753,-0.041366,-0.004521,-0.008163,-0.019790,-0.010659,...,0.009494,0.007121,-0.008891,0.003243,-0.001147,-0.000673,0.008342,-0.037654,-0.002246,-0.013605
5,2/23/2022 0:00,-0.017739,-0.025864,-0.025893,-0.035756,-0.069979,-0.017144,-0.014045,-0.017963,-0.042882,...,-0.020041,-0.002859,0.013066,-0.027287,0.006068,0.006991,0.013629,-0.008339,-0.025410,-0.042146
6,2/24/2022 0:00,0.015049,0.016680,0.051093,0.045095,0.048073,0.039996,0.039883,0.046107,0.060794,...,-0.017084,-0.024592,-0.045813,0.026071,-0.012062,0.017511,0.016087,-0.000647,-0.022937,0.035040
7,2/25/2022 0:00,0.022064,0.012966,0.009233,0.016058,0.011364,0.013328,0.013914,0.013873,0.017223,...,0.036092,0.032637,0.038531,0.014582,0.008414,0.034824,0.031393,0.023301,0.063503,0.032308
8,2/28/2022 0:00,-0.002558,0.001638,0.004978,-0.001466,0.074777,0.004444,0.002762,0.002613,0.009438,...,-0.016875,-0.019173,-0.003691,-0.011288,-0.011780,0.066660,-0.009180,-0.014969,-0.011910,-0.010183
9,3/1/2022 0:00,-0.015230,-0.011628,-0.012852,-0.015766,-0.006962,-0.007371,-0.005360,-0.035729,-0.037236,...,-0.060527,-0.017410,0.005459,-0.020837,-0.002318,0.052582,-0.002772,-0.047089,-0.065413,-0.033283


In [413]:
mu = discrete_return['META'].mean()
sigma = discrete_return['META'].std()
meta = discrete_return['META'].values - mu

In [414]:
weights = cum_weight(len(meta),lambda_=0.94)
sigma_exp = np.sqrt(sum(weights * meta * meta))

In [415]:
# since several method will use the ppf of normal distribution, use this function to calculate the results
def cal_var_norm(sigma, mean=0,alpha=0.05):
    return  -norm.ppf(alpha,loc=mean, scale=sigma)

In [416]:
# normal distribution with variance/exponential weighted variance

var_norm = cal_var_norm(sigma=sigma,alpha=0.05)


In [417]:
var_exp = cal_var_norm(sigma=sigma_exp,alpha=0.05)


In [423]:
# fit T distribution using MLE
df, mu_t, sigma_t = t.fit(meta, method='mle')
var_t = -t.ppf(0.05, df, mu_t,sigma_t)


In [419]:
# fit AR(1)

In [420]:
y_t = meta[1:].reshape(-1,1)
y_t_1 = meta[:-1].reshape(-1,1)
AR1 = LinearRegression().fit(y_t_1,y_t)
err = y_t - AR1.predict(y_t_1)
var_ar = cal_var_norm(sigma = err.std(), alpha= 0.05 ) + y_t[-1][0]


In [421]:
# Historical simulation
N = np.random.randint(len(meta), size=150)
his_sim = meta[N]
his_sim.sort()
var_hs = np.quantile(his_sim, 0.05)


In [424]:
f'{var_norm*100} % & {var_exp*100} & {var_t*100} & {var_ar*100} & {var_hs*100}'

'6.560156967533286 % & 9.138526093846897 & 5.757964975153305 & 3.5558174815379666 & -5.121255023873685'

# Problem 3

In [450]:
def cal_port(portfolio, prices, dateColumn="Date"):
    stocks = portfolio.loc[:,'Stock'].values
    prices_new = prices.loc[:,stocks]
    portfolio.loc[:,'price'] = [prices_new.loc[len(prices_new)-1, stock] for stock in stocks]
    value = (portfolio.Holding * portfolio.price).sum()
    prices_new.loc[:,dateColumn] = prices.loc[:,dateColumn].values
    return value, prices_new,portfolio


In [496]:
# calculate VaR using Pearson Covariance
def cal_port_var(port, dis="normal", simN=1000):
    value, prices, port = cal_port(port, price_data)
    return_dis = np.array(return_calculate(prices).drop('Date',axis=1))
    cov = np.cov(return_dis.T)

    if dis == "normal":
        sim_return = np.random.multivariate_normal(mean=np.zeros(len(cov)),cov=cov, size=simN)
    elif dis=="T":
        mle_t = lambda df: -sum(np.log(multivariate_t.pdf(return_dis,return_dis.mean(axis=0),cov, df=df))) if df>0 else -sum(np.log(multivariate_t.pdf(return_dis,return_dis.mean(axis=0),cov, df=df +1)))
        bnds = ((0, None),)
        res_t = minimize(mle_t, x0=1,bounds=bnds)

        df = res_t['x'][0]

        sim_return = multivariate_t.rvs(return_dis.mean(axis=0), cov,df=df, size=simN)
    var_dollor = -np.quantile(((sim_return)* port.loc[:,'price'].values * port.loc[:,'Holding'].values).sum(axis=1),0.05)
    var_pc = var_dollor/value * 100
    return var_dollor,var_pc

In [504]:
portfolio = pd.read_csv("portfolio.csv")
ports = [portfolio.loc[portfolio.Portfolio=='A'], portfolio.loc[portfolio.Portfolio=='B'],portfolio.loc[portfolio.Portfolio=='C'],portfolio]
for port in ports:
    var_dollar, var_pc = cal_port_var(port)
    var_dollar_t, var_pc_t = cal_port_var(port, dis="T")
    print(f'{round(var_dollar,4)} & { round(var_dollar_t,4)} \\\\')
    print(f'{round(var_pc,4)}\\% & { round(var_pc_t,4)}\\%  \\\\')


7862.6238 & 8568.246 \\
2.6213\% & 2.8566\%  \\
6637.4965 & 7573.7378 \\
2.2547\% & 2.5727\%  \\
5752.6525 & 6724.801 \\
2.1303\% & 2.4903\%  \\
19311.4454 & 19719.8069 \\
2.2341\% & 2.2814\%  \\


In [511]:
def cal_port_var_ew(port, dis="normal", simN=1500):
    # calculate VaR using EW Covariance
    value, prices, port = cal_port(port, price_data)
    return_dis = return_calculate(prices)

    cov = ew_cov(return_dis.values[:,1:], lambda_=0.94)
    if dis == "normal":
        sim_return = np.random.multivariate_normal(mean=np.zeros(len(cov)),cov=cov, size=simN)
    elif dis=="T":

        mle_t = lambda df: -sum(np.log(multivariate_t.pdf(return_dis.values[:,1:],return_dis.mean(),cov, df=df))) if df>0 else -sum(np.log(multivariate_t.pdf(return_dis.values[:,1:],return_dis.mean(), cov, df=df +1)))
        bnds = ((0, None),)
        res_t = minimize(mle_t, x0=1,bounds=bnds)
        df = res_t['x'][0]
        sim_return = multivariate_t.rvs(return_dis.drop("Date",axis=1).mean().values,cov,df=df, size=simN)

    var_dollar = -np.quantile(((sim_return)* port.loc[:,'price'].values * port.loc[:,'Holding'].values).sum(axis=1),0.05)
    var_pc = var_dollar/value * 100
    return var_dollar,var_pc


In [512]:
for port in ports:
    var_dollar, var_pc = cal_port_var_ew(port)
    var_dollar_t, var_pc_t = cal_port_var_ew(port, dis="T")
    print(f'{var_dollar} & { var_dollar_t}  \\\\')
    print(f'{var_pc}\\% & { var_pc_t}\\%  \\\\')

5476.7251036327125 & 9681.964451334687  \\
1.8258789881696837\% & 3.2278588246417454\%  \\
4385.863896084081 & 6892.211111205705  \\
1.4898364705631255\% & 2.3412189068298925\%  \\
3723.396147406015 & 7153.515417519049  \\
1.378816886243563\% & 2.6490299348218644\%  \\
13491.658114820217 & 224320.8904777282  \\
1.5608507639242137\% & 25.95169772955573\%  \\
